正样本处理
裁剪行人区域至统一尺寸（64*128），居中对齐

In [ ]:
import cv2
import os

from sympy.codegen.ast import continue_
from torch_geometric.typing import weight

current_dir=os.getcwd()
print(current_dir)

# 定义一个函数，用于处理正样本图片
def process_positive_images(src_dir,dst_dir,img_size=(64,128)):
    # 创建目标文件夹
    os.makedirs(dst_dir,exist_ok=True)
    # 遍历源文件夹中的所有文件
    for filename in os.listdir(src_dir):
        # 获取图片路径
        img_path=os.path.join(src_dir,filename)
        # 读取图片
        img=cv2.imread(img_path)
        # 如果图片为空，则跳过
        if img is None:
            continue
        # 获取图片的高度和宽度
        h,w=img.shape[:2]
        # 获取裁剪的高度和宽度
        crop_h,crop_w=img_size
        # 计算裁剪的起始高度和宽度
        start_h=max(0,(h-crop_h)//2)
        start_w=max(0,(w-crop_w)//2)
        # 裁剪图片
        cropped=img[start_h:start_h+crop_h,start_w:start_w+crop_w]
        # 将裁剪后的图片保存到目标文件夹中
        cv2.imwrite(os.path.join(dst_dir,filename),cropped)

process_positive_images("./train/pos", "./Train/pos64x128", img_size=(64, 128))

负样本增强
从背景图像中随机裁剪10个非重叠区域，扩充负样本数量

In [ ]:
import random

def process_negative_images(src_dir,dst_dir,img_size=(64,128),random_num=10):
    # 创建目标目录
    os.makedirs(dst_dir,exist_ok=True)
    idx=0
    # 遍历源目录中的所有文件
    for filename in os.listdir(src_dir):
        img_path=os.path.join(src_dir,filename)
        # 读取图片
        img=cv2.imread(img_path)
        # 如果图片读取失败，则跳过
        if img is None:
            continue
        # 获取图片的高度和宽度
        h,w=img.shape[:2]
        # 获取裁剪的高度和宽度
        crop_h,crop_w=img_size
        # 如果图片的高度或宽度小于裁剪的高度或宽度，则跳过
        if h<crop_h or w<crop_w:
            continue
        # 随机裁剪图片
        for _ in range(random_num):
            # 随机生成裁剪的起始位置
            start_h=random.randint(0,h-crop_h)
            start_w=random.randint(0,w-crop_w)
            # 裁剪图片
            cropped=img[start_h:start_h+crop_h,start_w:start_w+crop_w]
            # 保存裁剪后的图片
            cv2.imwrite(os.path.join(dst_dir,f"{idx}.jpg"),cropped)
            idx+=1

process_negative_images("./train/neg", "./Train/neg64x128", img_size=(64, 128),random_num=10)

直接调用OpenCv内置的HOG+SVM
这里要注意opencv的版本，低版本没有检测路人的模块


In [45]:
import cv2

print(cv2.__version__)
def detect_persons_hog_svm(image_path):
    hog=cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    #读取照片
    img=cv2.imread(image_path)
    if img is None:
        raise ValueError("无法读取图片")
    rects,weights=hog.detectMultiScale(
        img,
        winStride=(4,4),
        padding=(8,8),
        scale=1.05
    )
    for (x,y,w,h) in rects:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.imshow("img",img)
    cv2.waitKey(0)
detect_persons_hog_svm("./1.jpg")

4.12.0
